In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.936641,False,30,1,38.0
1,8.624484,True,34,1,39.0
2,6.937947,True,23,1,39.0
3,6.935743,True,18,1,39.0
4,8.318041,False,23,1,40.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9995.000000,10000.0000,10000.000000,9885.000000
mean,7.263349,27.1810,1.035300,38.705311
std,1.315565,6.1537,0.196616,2.567750
min,0.507063,12.0000,1.000000,18.000000
25%,6.624891,22.0000,1.000000,38.000000
50%,7.374463,27.0000,1.000000,39.000000
75%,8.062305,32.0000,1.000000,40.000000
max,12.625874,48.0000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5198
False    4802
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,0,33,1,38.0
6130,0,20,1,39.0
5986,1,37,1,33.0
7681,1,28,1,39.0
4913,0,27,1,39.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 6ms/step - loss: 2.0077 - mae: 1.0361 - mse: 2.0077 - val_loss: 2.5252 - val_mae: 1.3017 - val_mse: 2.5252
Epoch 2/10
209/209 [==============================] - 1s 5ms/step - loss: 1.4966 - mae: 0.9569 - mse: 1.4966 - val_loss: 1.2673 - val_mae: 0.8710 - val_mse: 1.2673
Epoch 3/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4168 - mae: 0.9264 - mse: 1.4168 - val_loss: 1.4089 - val_mae: 0.9224 - val_mse: 1.4089
Epoch 4/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3840 - mae: 0.9200 - mse: 1.3840 - val_loss: 2.3940 - val_mae: 1.2715 - val_mse: 2.3940
Epoch 5/10
209/209 [==============================] - 1s 4ms/step - loss: 1.3280 - mae: 0.8937 - mse: 1.3280 - val_loss: 1.4892 - val_mae: 0.9532 - val_mse: 1.4892
Epoch 6/10
209/209 [==============================] - 1s 5ms/step - loss: 1.3049 - mae: 0.8862 - mse: 1.3049 - val_loss: 1.2298 - val_mae: 0.8615 - val_mse: 1.2298
Epoch 7/10
209/2

In [12]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  5.5381103
Actual val:  3.1415872335

Predicted val:  7.2187057
Actual val:  6.3118345610599995

Predicted val:  7.6234465
Actual val:  12.62587374474

Predicted val:  7.280079
Actual val:  7.3744626639

Predicted val:  7.259196
Actual val:  7.1760466281

Predicted val:  7.4661117
Actual val:  5.62619692624

Predicted val:  7.64409
Actual val:  7.12534030784

Predicted val:  0.775372
Actual val:  0.8377565955999999

Predicted val:  7.5942874
Actual val:  7.3965088901

Predicted val:  7.059657
Actual val:  6.56316153974



In [14]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [15]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [16]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…